# Change Composition Using MDAnalysis
We will use the following outline:
1. Import all necessary packages
2. Randomly select a specified number of DPPC
3. Edit those DPPC, preferably using MDAnalysis
4. Export using mdanalysis.

In [14]:
import MDAnalysis as mda
import sys
import random

input_file = sys.argv[1] #Must be a .gro file
outfile = sys.argv[2] #Must be a name, not a file with extension. Will be exported as gro.

# input_file = "dppc_bilayer.gro"
bilayer = mda.Universe(input_file)

# Randomly select 35 lipids in range [1,129)
rlist = random.sample(xrange(129), 35)
sel = bilayer.select_atoms("resid 0") #This doesn't select anything, it just defines sel as an empty atom selection

for i in rlist:
    sel = sel + bilayer.select_atoms("resid %s" % i)

# Rename these lipids to DBPC
sel.resnames = "DBPC"

# Select all DPPC and W to transfer to output file
DPPW = bilayer.select_atoms("resname DPPC") + bilayer.select_atoms("resname W")

# Select the atoms from DBPC that actually belong to all DBPC molecules. IE, chop molecules by not selecting them.
rm = bilayer.select_atoms("resname DBPC and not (name C2A or name C3A or name C4A or name C2B or name C3B or name C4B)")

# Combine the unedited DPPC and W with the edited DBPC molecules. Then convert this to an atom list and write it to a gro file.
new = mda.Merge(rm, DPPW)
new = new.atoms
new.write(filename=outfile, format="GRO")

### FOR SOME REASON THIS IS INCAPABLE OF PRINTING ACTUAL BOX COORDINATES. YOU HAVE TO DO IT MANUALLY. FIX THIS!




In [8]:
import MDAnalysis as mda
bilayer = mda.Universe("dppc_bilayer.gro")
bilayeratoms = bilayer.atoms
x = mda.coordinates.GRO.GROWriter(filename="TESTGRO").fmt['box_orthorhombic'].format(box=(25.0, 25.0 , 10.0))
bilayeratoms.write(x)

ValueError: No writer found for format:   25.00000  25.00000  10.00000
